### Дисперсионный анализ

**Задача:   
Даны заработные платы юристов, программистов и бухгалтеров. Определить, влияет ли профессия на заработную плату!**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
import pandas as pd

In [3]:
y1 = np.array([70, 50, 65, 60, 75, 67, 74])         # зарплата бухгалтеров
y2 = np.array([80, 74, 90, 70, 75, 65, 85])         # зарплата юристов
y3 = np.array([148, 142, 140, 150, 160, 170, 155])  # зарплата программистов

In [4]:
k = 3    # количество сравниваемых групп
n = 21   # общее количество значений во всех группах

In [5]:
y_mean_1 = np.mean(y1)  # среднее арифметическое для зарплаты бухгалтеров
y_mean_1

65.85714285714286

In [6]:
y_mean_2 = np.mean(y2)  # среднее арифметическое для зарплаты юристов
y_mean_2

77.0

In [7]:
y_mean_3 = np.mean(y3)  # среднее арифметическое для зарплаты программистов
y_mean_3

152.14285714285714

In [10]:
total = np.array([y1, y2, y3], dtype=object)  # объединяем три массива в один
total

array([[70, 50, 65, 60, 75, 67, 74],
       [80, 74, 90, 70, 75, 65, 85],
       [148, 142, 140, 150, 160, 170, 155]], dtype=object)

In [11]:
y_mean_total = np.mean(total)
y_mean_total

98.33333333333333

Сумма квадратов отклонений наблюдений от общего среднего:

In [36]:
np.sum((total - y_mean_total) ** 2)   # отложим это значение

32400.66666666667

Сумма квадратов отклонений средних групповых значений от общего среднего (фактическая):

In [37]:
S_f = np.sum((y_mean_1 - y_mean_total) ** 2) * 7 + np.sum((y_mean_2 - y_mean_total) ** 2) * 7 + np.sum((y_mean_3 - y_mean_total) ** 2) * 7
S_f

30836.952380952374

Остаточная сумма квадратных отклонений: 

In [39]:
S_ost = np.sum((y1 - y_mean_1) ** 2) + np.sum((y2 - y_mean_2) ** 2) + np.sum((y3 - y_mean_3) ** 2)
S_ost

1563.7142857142858

Проверим: Общая сумма квадратных отклонений должна быть равна сумме фактических и останочных квадратных отклонений:

In [41]:
print(S_f + S_ost)
np.sum((total - y_mean_total) ** 2)

32400.66666666666


32400.66666666667

Найдем фактическую и останочную дисперсии:    
$\sigma_{факт}^2 = \frac{S_{факт}^2}{k-1}$    
$\sigma_{ост}^2 = \frac{S_{ост}^2}{n-k}$

In [42]:
D_f = S_f/(k - 1)
D_f

15418.476190476187

In [43]:
D_ost = S_ost/(n - k)
D_ost

86.87301587301587

Найдем наблюдаемый коэффициент Фишера:   
$F_н = \frac{\sigma_{факт}^2}{\sigma_{ост}^2}$

In [45]:
F_n = D_f/D_ost
F_n

177.48291613374744

Табличный коэффициент Фишера находим исходя из степеней свободы числитея (2) и знаменателя (18) равен 3,55 

Воспользовавшись встроенной функцией для определения коэффициента Фишера

In [47]:
f = stats.f_oneway(y1, y2, y3)
f

F_onewayResult(statistic=177.48291613374704, pvalue=1.420466900107174e-12)

видим, что p-value меньше 0,05, поэтому делаем заключение, что существкет статистически значимые отличия между заработными платами рассматриваемых профессий.  

*Но мы не знаем между какими именно профессиями существуют различия*

**Post hoc test Tukey!**    
Воспользуемся Post hoc тестом Тьюка

In [50]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [52]:
# Создадит дата фрэйм для вывода результатов в виде таблицы
df= pd.DataFrame({'score': [70, 50, 65, 60, 75, 67, 74,
                            80, 74, 90, 70, 75, 65, 85,
                            148, 142, 140, 150, 160, 170, 155],
                  'group': np.repeat(['acountant', 'lawyer', 'programmer'], repeats=7)})
tukey = pairwise_tukeyhsd(endog=df['score'],
                          groups=df['group'],
                          alpha=0.05)
print(tukey)

    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
  group1    group2   meandiff p-adj   lower   upper  reject
-----------------------------------------------------------
acountant     lawyer  11.1429 0.0918 -1.5722 23.8579  False
acountant programmer  86.2857    0.0 73.5707 99.0007   True
   lawyer programmer  75.1429    0.0 62.4278 87.8579   True
-----------------------------------------------------------


Видим, что статистически значимых отличий между зарплатами бухгалтеров и юристов нет, однако отличия есть между зарплатой программистов и бухгалтеров и программистов и юристов.

**Духфакторый дисперсионный анализ**

*Пример с кондиционерами.*   
Две детали в кондиционерах, которые конструктора определили ключевыми факторами, влияющими на производительности продукции. Предположим, мы будем измерять количество квадратных метров, охлажденных за какой-то определенный промежуток времени. Первая и вторая деталь  - это фактор A и фактор B соответственно. Те размеры деталей, с которыми сейчас изготавливают кондиционер, называются первым уровнем, а следующий размер для каждой детали – это 2й уровень. Для простоты мы рассмотрим дизайн эксперимента 2 Х 2. Есть 2 детали и для каждой из них есть 2 уровня. Мы будем комбинировать эти уровни и замерять производительность. Для каждой комбинации проводим эксперимент  не менее 2х раз.

Первая комбинация – это 1й уровень детали A и 1й уровень детали B. Помним, что измерения проводятся минимум 2 раза. Мы для каждой комбинации проведем опыт по 2 раза. Первый опыт при уровнях 1-1 дал значение $57 м^2$, а при повторном измерении на этих же уровнях $59 м^2$. Среднее арифметическое для них 58.
Давайте оформим эти комбинации уровней и значений производительности в  виде таблице.  Значения 57,  59 и их среднее 58 – это 1я ячейка со значениями. Значения 57.5 и 52 – это средние по срокам, а 45.5 и 64 – это средние по столбцам. 54.75 – это среднее по всем значениям.


Запишем значения переменных и расситаем все значения:

In [53]:
y111 = 57   # значение первого эксперимента
y112 = 59   # значение второго эксперимента 
y11 = (y111 + y112) / 2 # среднее
y11

58.0

In [55]:
y121 = 56
y122 = 58
y12 = (y121 + y122) / 2
y12

57.0

In [57]:
y211 = 32
y212 = 34
y21 = (y211 + y212) / 2
y21

33.0

In [65]:
y221 = 71
y222 = 71
y22 = (y221 + y222) / 2
y22

71.0

In [66]:
YcpA1 = (y11 + y12) / 2  # среднее для перого уровня A
YcpA1

57.5

In [67]:
YcpA2 = (y21 + y22) / 2  # среднее для второго уровня A
YcpA2

52.0

In [68]:
YcpB1 = (y11 + y21) / 2  # среднее для перого уровня B
YcpB1

45.5

In [69]:
YcpB2 = (y12 + y22) / 2 # среднее для второго уровня A
YcpB2

64.0

In [71]:
Ycp = np.mean(YcpA1 + YcpA2 + YcpB1 + YcpB2) / 4
Ycp

54.75

*Теперь будем делать расчеты и заносить их результаты в **ANOVA** таблицу*

In [73]:
a = 2  # два уровня фактора a
b = 2  # два уровня фактора b
n = k = 2 # число повторных измерений

In [78]:
SSt = (y111 ** 2)+(y112 ** 2)+(y121 ** 2)+(y122 ** 2)+(y211 ** 2)+(y212 ** 2)+(y221 ** 2)+(y222 ** 2) - a*b*n*(Ycp**2)
SSt

1511.5

In [82]:
SSA = a*n*(YcpA1**2 + YcpA2**2) - a*b*n*(Ycp)**2
SSA

60.5

In [83]:
SSB = a*n*(YcpB1**2 + YcpB2**2) - a*b*n*(Ycp)**2
SSB

684.5

In [87]:
SSAB = n*(y11**2 + y12**2 + y21**2 + y22**2) - a*b*n*(Ycp)**2 - SSA - SSB
SSAB

760.5

In [88]:
SSE = SSt - SSA - SSB - SSAB
SSE

6.0

Рассчитаем теперь степени свободы *df* и сумму квадратов отклонений в расчете на одну степень свободы.

In [98]:
dfA = a - 1
dfB = b - 1
dfA, dfB

(1, 1)

In [99]:
dfAB = (a - 1)/(b - 1)
dfE = a * b * (n -1)
dfAB, dfE

(1.0, 4)

In [100]:
MSA = SSA / dfA
MSB = SSB / dfB
MSA, MSB

(60.5, 684.5)

In [97]:
MSAB = SSAB / dfAB
MSE = SSE / dfE
MSAB, MSE

(760.5, 1.5)

In [96]:
FA = MSA / MSE 
FB = MSB / MSE 
FAB = MSAB / MSE
FA, FB, FAB

(40.333333333333336, 456.3333333333333, 507.0)

Теперь осталось рассчитать для каждого фактора и взаимодействия факторов критерий Фишера. Расчеты критериев мы сделаем ниже. Значения получим следующие: $F_A=40.33,F_B=456.33,F_AB=507$. И также для каждого нужно найти  табличный критерий Фишера для α = 0.05 и степеней свободы 1 и 4.  Как видите, степени свободы совпадают для факторов и их взаимодействия (ниже разберем, откуда 1 и 4), но так происходит именно в нашем случае, т.к. эксперимент 2 Х 2.    
*Откуда 1 и 4 степеней свободы?*   
Например, критерий Фишера для фактора А рассчитываем следующим  образом:
$F =  MSA /  MSE$ . Когда мы искали MSA – мы $SS_A$ делили на 1 степень свободы dfA . 1 – это степень свободы числителя и в таблице находятся в горизонтальной строке.  А, когда находили MSE,то  $SS_E$ делили на 4 степени свободы. 4 –это степени свободы знаменателя и находятся в вертикальной строке таблицы. По тому же принципу степени свободы критерия Фишера для фактора B и взаимодействия факторов АB будут 1 и 4.  Т.о. в нашем примере $F_т$  для всех факторов и их взаимодействия равен 7.71


Сейчас мы создадим уровни фактора A, уровни фактора B –массивы  fA и  fB.
Массив values  - это значения площади, которые соответствуют комбинации уровней.

In [101]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [102]:
fA = np.array(['low', 'low', 'low', 'low', 'high', 'high', 'high', 'high'])
fA

array(['low', 'low', 'low', 'low', 'high', 'high', 'high', 'high'],
      dtype='<U4')

In [103]:
fB = np.array(['low', 'low', 'high', 'high', 'low', 'low', 'high', 'high'])
fB

array(['low', 'low', 'high', 'high', 'low', 'low', 'high', 'high'],
      dtype='<U4')

In [104]:
values = np.array([57, 59, 56, 58, 32, 34, 71, 71])
values

array([57, 59, 56, 58, 32, 34, 71, 71])

Построим таблицу эксперимента 2 Х 2

In [105]:
df=pd.DataFrame({'fA': fA, 'fB': fB, 'values': values})
df

,fA,fB,values
0,low,low,57
1,low,low,59
2,low,high,56
3,low,high,58
4,high,low,32
5,high,low,34
6,high,high,71
7,high,high,71


Теперь пришло время строить **ANOVA** таблицу, но сначала построим математическую модель *lm_model*. 

In [107]:
lm_model = ols('values ~ C(fA) * C(fB)', data = df).fit()
#строим ANOVA таблицу
table = sm.stats.anova_lm(lm_model, typ=2)
table

,sum_sq,df,F,PR(>F)
C(fA),60.5,1.0,40.333333,0.003150
C(fB),684.5,1.0,456.333333,0.000028
C(fA):C(fB),760.5,1.0,507.000000,0.000023
Residual,6.0,4.0,NaN,NaN
